In [1]:
import json
import pandas as pd

weatherdf = pd.read_json("extracted_data/historical_weather_kelvins.json")
tracksdf = pd.read_json("extracted_data/tracks_wids_nozeroes.json")
featuresdf = pd.read_json("extracted_data/raw_audio_features.json")     

In [2]:
def kelvin2fahrenheit(k):
    f = k - 273.15
    f *= 1.8
    f += 32
    return round(f, 2)

def milli_to_min(milli):
    seconds = milli/1000
    minutes = seconds/60
    return round(minutes, 2)


## Transform Weather Data

In [3]:
weatherdf.head()

,lat,lon,tz,date,units,cloud_cover,humidity,precipitation,temperature,pressure,wind
0,40.6596,-111.9193,-06:00,2023-03-18,standard,{'afternoon': 0.0},{'afternoon': 25.0},{'total': 0.0},"{'min': 270.4, 'max': 282.8, 'afternoon': 279....",{'afternoon': 1026.0},"{'max': {'speed': 6.71, 'direction': 168.0}}"
1,40.6596,-111.9193,-06:00,2023-03-19,standard,{'afternoon': 0.0},{'afternoon': 26.0},{'total': 0.33},"{'min': 274.29, 'max': 284.93, 'afternoon': 28...",{'afternoon': 1017.0},"{'max': {'speed': 10.8, 'direction': 180.0}}"
2,40.6596,-111.9193,-06:00,2023-03-20,standard,{'afternoon': 0.0},{'afternoon': 74.0},{'total': 9.65},"{'min': 274.41, 'max': 281.42, 'afternoon': 28...",{'afternoon': 1006.0},"{'max': {'speed': 14.92, 'direction': 300.0}}"
3,40.6596,-111.9193,-06:00,2023-03-21,standard,{'afternoon': 0.0},{'afternoon': 71.0},{'total': 2.04},"{'min': 272.25, 'max': 282.11, 'afternoon': 27...",{'afternoon': 1008.0},"{'max': {'speed': 5.66, 'direction': 160.0}}"
4,40.6596,-111.9193,-06:00,2023-03-22,standard,{'afternoon': 100.0},{'afternoon': 69.0},{'total': 2.24},"{'min': 275.85, 'max': 278.64, 'afternoon': 27...",{'afternoon': 1003.0},"{'max': {'speed': 11.83, 'direction': 180.0}}"


Extract nested values from columns in dataframe. (i.e. get 25.0 from {'afternoon': 25.0})

In [4]:
weather_norm = pd.DataFrame()
weather_norm['date'] = weatherdf['date']
weather_norm['cloud_coverage'] = weatherdf['cloud_cover'].apply(lambda x: x.get('afternoon'))
weather_norm['humidity'] = weatherdf['humidity'].apply(lambda x: x.get('afternoon'))
weather_norm['precipitation_total'] = weatherdf['precipitation'].apply(lambda x: x.get('total'))
weather_norm['min_temperature'] = weatherdf['temperature'].apply(lambda x: x.get('min'))
weather_norm['max_temperature'] = weatherdf['temperature'].apply(lambda x: x.get('max'))
weather_norm['barometric_pressure'] = weatherdf['pressure'].apply(lambda x: x.get('afternoon'))
weather_norm['wind_max_speed'] = weatherdf['wind'].apply(lambda x: x['max']['speed'])
weather_norm['wind_direction'] = weatherdf['wind'].apply(lambda x: x['max']['direction'])
weather_norm.head(10)

,date,cloud_coverage,humidity,precipitation_total,min_temperature,max_temperature,barometric_pressure,wind_max_speed,wind_direction
0,2023-03-18,0.0,25.0,0.00,270.40,282.80,1026.0,6.71,168.0
1,2023-03-19,0.0,26.0,0.33,274.29,284.93,1017.0,10.80,180.0
2,2023-03-20,0.0,74.0,9.65,274.41,281.42,1006.0,14.92,300.0
3,2023-03-21,0.0,71.0,2.04,272.25,282.11,1008.0,5.66,160.0
4,2023-03-22,100.0,69.0,2.24,275.85,278.64,1003.0,11.83,180.0
5,2023-03-23,75.0,83.0,3.46,274.74,278.64,1018.0,6.69,160.0
6,2023-03-24,100.0,90.0,13.20,271.21,275.08,1016.0,11.83,290.0
7,2023-03-25,100.0,49.0,3.97,269.24,274.35,1020.0,10.80,260.0
8,2023-03-26,100.0,76.0,7.47,267.50,273.85,1020.0,10.29,320.0
9,2023-03-27,100.0,73.0,4.30,269.52,276.28,1030.0,5.66,240.0


Transform the temperature readings by converting them from kelvin units to fahrenheit units. 

In [5]:
weather_imperial = pd.DataFrame(weather_norm)
weather_imperial['min_temperature'] = kelvin2fahrenheit(weather_norm['min_temperature'])
weather_imperial['max_temperature'] = kelvin2fahrenheit(weather_norm['max_temperature'])
weather_imperial.head(10)

,date,cloud_coverage,humidity,precipitation_total,min_temperature,max_temperature,barometric_pressure,wind_max_speed,wind_direction
0,2023-03-18,0.0,25.0,0.00,27.05,49.37,1026.0,6.71,168.0
1,2023-03-19,0.0,26.0,0.33,34.05,53.20,1017.0,10.80,180.0
2,2023-03-20,0.0,74.0,9.65,34.27,46.89,1006.0,14.92,300.0
3,2023-03-21,0.0,71.0,2.04,30.38,48.13,1008.0,5.66,160.0
4,2023-03-22,100.0,69.0,2.24,36.86,41.88,1003.0,11.83,180.0
5,2023-03-23,75.0,83.0,3.46,34.86,41.88,1018.0,6.69,160.0
6,2023-03-24,100.0,90.0,13.20,28.51,35.47,1016.0,11.83,290.0
7,2023-03-25,100.0,49.0,3.97,24.96,34.16,1020.0,10.80,260.0
8,2023-03-26,100.0,76.0,7.47,21.83,33.26,1020.0,10.29,320.0
9,2023-03-27,100.0,73.0,4.30,25.47,37.63,1030.0,5.66,240.0


round data to 2 decimal places

In [6]:
weather_imperial = weather_imperial.round({'cloud_coverage': 2, 'humidity': 2,'precipitation_total': 2,'min_temperature': 2,'max_temperature': 2,'barometric_pressure': 2,'wind_max_speed': 2,'wind_direction': 2})
weather_imperial.head(10)

,date,cloud_coverage,humidity,precipitation_total,min_temperature,max_temperature,barometric_pressure,wind_max_speed,wind_direction
0,2023-03-18,0.0,25.0,0.00,27.05,49.37,1026.0,6.71,168.0
1,2023-03-19,0.0,26.0,0.33,34.05,53.20,1017.0,10.80,180.0
2,2023-03-20,0.0,74.0,9.65,34.27,46.89,1006.0,14.92,300.0
3,2023-03-21,0.0,71.0,2.04,30.38,48.13,1008.0,5.66,160.0
4,2023-03-22,100.0,69.0,2.24,36.86,41.88,1003.0,11.83,180.0
5,2023-03-23,75.0,83.0,3.46,34.86,41.88,1018.0,6.69,160.0
6,2023-03-24,100.0,90.0,13.20,28.51,35.47,1016.0,11.83,290.0
7,2023-03-25,100.0,49.0,3.97,24.96,34.16,1020.0,10.80,260.0
8,2023-03-26,100.0,76.0,7.47,21.83,33.26,1020.0,10.29,320.0
9,2023-03-27,100.0,73.0,4.30,25.47,37.63,1030.0,5.66,240.0


## Transform streaming history data

In [45]:
tracksdf.head()

,endTime,artistName,trackName,msPlayed,trackId,streamingDate
0,1679174880000,Eudes Alvarez,San Rafael de Mi Pueblo,16688,7A80n2HvAs1rZNNs64NN8F,2023-03-18
1,1679183520000,Sam Cooke,Another Saturday Night,153052,5z8DiKSG4EwxmO6gUi12rZ,2023-03-18
3,1679254440000,Soledad Bravo,Violín de becho,13552,7hkyFS309jQBrwrbxGsJuH,2023-03-19
4,1679254620000,Soledad Bravo,Violín de becho,196288,7hkyFS309jQBrwrbxGsJuH,2023-03-19
5,1679254740000,Soledad Bravo,Canción para mi américa,113226,54a3QojIm00P4MXL02BZvC,2023-03-19


In [46]:
tracksdf2 = pd.DataFrame(tracksdf)
tracksdf2['msPlayed'] = milli_to_min(tracksdf2['msPlayed'])
tracksdf2 = tracksdf2.rename(columns={'msPlayed': 'minutesPlayed'})
tracksdf2.drop('endTime', axis=1, inplace=True)
tracksdf2.head()

,artistName,trackName,minutesPlayed,trackId,streamingDate
0,Eudes Alvarez,San Rafael de Mi Pueblo,0.28,7A80n2HvAs1rZNNs64NN8F,2023-03-18
1,Sam Cooke,Another Saturday Night,2.55,5z8DiKSG4EwxmO6gUi12rZ,2023-03-18
3,Soledad Bravo,Violín de becho,0.23,7hkyFS309jQBrwrbxGsJuH,2023-03-19
4,Soledad Bravo,Violín de becho,3.27,7hkyFS309jQBrwrbxGsJuH,2023-03-19
5,Soledad Bravo,Canción para mi américa,1.89,54a3QojIm00P4MXL02BZvC,2023-03-19


## Transform audio features data

In [3]:
featuresdf.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.375,0.728,9,-10.130,0,0.0433,0.759,0.860000,0.1410,0.9010,118.414,audio_features,7A80n2HvAs1rZNNs64NN8F,spotify:track:7A80n2HvAs1rZNNs64NN8F,https://api.spotify.com/v1/tracks/7A80n2HvAs1r...,https://api.spotify.com/v1/audio-analysis/7A80...,177800,4
1,0.776,0.659,9,-7.767,1,0.0510,0.434,0.000000,0.0868,0.9690,123.094,audio_features,5z8DiKSG4EwxmO6gUi12rZ,spotify:track:5z8DiKSG4EwxmO6gUi12rZ,https://api.spotify.com/v1/tracks/5z8DiKSG4Ewx...,https://api.spotify.com/v1/audio-analysis/5z8D...,160000,4
2,0.426,0.300,11,-9.717,0,0.0365,0.573,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,spotify:track:7hkyFS309jQBrwrbxGsJuH,https://api.spotify.com/v1/tracks/7hkyFS309jQB...,https://api.spotify.com/v1/audio-analysis/7hky...,209840,4
3,0.426,0.300,11,-9.717,0,0.0365,0.573,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,spotify:track:7hkyFS309jQBrwrbxGsJuH,https://api.spotify.com/v1/tracks/7hkyFS309jQB...,https://api.spotify.com/v1/audio-analysis/7hky...,209840,4
4,0.555,0.293,3,-8.525,0,0.0470,0.877,0.000001,0.1180,0.1130,140.935,audio_features,54a3QojIm00P4MXL02BZvC,spotify:track:54a3QojIm00P4MXL02BZvC,https://api.spotify.com/v1/tracks/54a3QojIm00P...,https://api.spotify.com/v1/audio-analysis/54a3...,113227,3
5,0.614,0.277,9,-12.052,1,0.1200,0.852,0.000000,0.1060,0.5810,122.659,audio_features,1ebslsQcXFPNRBLsa76qEK,spotify:track:1ebslsQcXFPNRBLsa76qEK,https://api.spotify.com/v1/tracks/1ebslsQcXFPN...,https://api.spotify.com/v1/audio-analysis/1ebs...,180082,4
6,0.137,0.119,1,-23.390,1,0.0575,0.978,0.940000,0.1010,0.0374,70.023,audio_features,41F8XFf9aYq1vCMqyuLNFl,spotify:track:41F8XFf9aYq1vCMqyuLNFl,https://api.spotify.com/v1/tracks/41F8XFf9aYq1...,https://api.spotify.com/v1/audio-analysis/41F8...,336001,4
7,0.818,0.151,9,-21.946,0,0.0718,0.917,0.917000,0.1110,0.4140,142.153,audio_features,6OzRUo7b23kXFXHWLpigCj,spotify:track:6OzRUo7b23kXFXHWLpigCj,https://api.spotify.com/v1/tracks/6OzRUo7b23kX...,https://api.spotify.com/v1/audio-analysis/6OzR...,121690,4
8,0.566,0.237,6,-17.012,0,0.0282,0.398,0.897000,0.1160,0.0761,80.020,audio_features,61liR3eWrA8BTeUnltA32e,spotify:track:61liR3eWrA8BTeUnltA32e,https://api.spotify.com/v1/tracks/61liR3eWrA8B...,https://api.spotify.com/v1/audio-analysis/61li...,147000,4
9,0.694,0.337,5,-12.300,0,0.0624,0.609,0.597000,0.1760,0.5690,126.235,audio_features,601ppwJ942tjJ9aDPGYwOT,spotify:track:601ppwJ942tjJ9aDPGYwOT,https://api.spotify.com/v1/tracks/601ppwJ942tj...,https://api.spotify.com/v1/audio-analysis/601p...,190476,4


Use the map function to convert numerical values into descriptive values for key signature, mode, and time signature.

In [4]:
key_sigs = {0:'C', 1:'C#/Db', 2:'D', 3:'D#/Eb', 4:'E', 5:'F', 6:'F#/Gb', 7:'G', 8:'G#/Ab', 9:'A', 10:'A#/Bb', 11:'B'}
modes = {0:'minor', 1:'major'}
time_sigs = {3:'3/4', 4:'4/4', 5:'5/4', 6:'6/4', 7:'7/4'}
features2 = pd.DataFrame(featuresdf)
features2['key'] = features2['key'].map(key_sigs)
features2['mode'] = features2['mode'].map(modes)
features2['time_signature'] = features2['time_signature'].map(time_sigs)
columns_to_drop = ['uri', 'track_href', 'analysis_url']
features2.drop(columns=columns_to_drop, inplace=True)

features2.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,duration_ms,time_signature
0,0.375,0.728,A,-10.130,minor,0.0433,0.759,0.860000,0.1410,0.9010,118.414,audio_features,7A80n2HvAs1rZNNs64NN8F,177800,4/4
1,0.776,0.659,A,-7.767,major,0.0510,0.434,0.000000,0.0868,0.9690,123.094,audio_features,5z8DiKSG4EwxmO6gUi12rZ,160000,4/4
2,0.426,0.300,B,-9.717,minor,0.0365,0.573,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,209840,4/4
3,0.426,0.300,B,-9.717,minor,0.0365,0.573,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,209840,4/4
4,0.555,0.293,D#/Eb,-8.525,minor,0.0470,0.877,0.000001,0.1180,0.1130,140.935,audio_features,54a3QojIm00P4MXL02BZvC,113227,3/4
5,0.614,0.277,A,-12.052,major,0.1200,0.852,0.000000,0.1060,0.5810,122.659,audio_features,1ebslsQcXFPNRBLsa76qEK,180082,4/4
6,0.137,0.119,C#/Db,-23.390,major,0.0575,0.978,0.940000,0.1010,0.0374,70.023,audio_features,41F8XFf9aYq1vCMqyuLNFl,336001,4/4
7,0.818,0.151,A,-21.946,minor,0.0718,0.917,0.917000,0.1110,0.4140,142.153,audio_features,6OzRUo7b23kXFXHWLpigCj,121690,4/4
8,0.566,0.237,F#/Gb,-17.012,minor,0.0282,0.398,0.897000,0.1160,0.0761,80.020,audio_features,61liR3eWrA8BTeUnltA32e,147000,4/4
9,0.694,0.337,F,-12.300,minor,0.0624,0.609,0.597000,0.1760,0.5690,126.235,audio_features,601ppwJ942tjJ9aDPGYwOT,190476,4/4


Convert song duration time from milliseconds to minutes and rename column to match data.

In [5]:
features2 = features2.rename(columns={'duration_ms': 'length_in_minutes'})
features2['length_in_minutes'] = milli_to_min(features2['length_in_minutes'])
# features2 = features2.round({'danceability': 2, 'energy': 2,'loudness': 2,'speechiness': 2,'acousticness': 2,'instrumentalness': 2,'liveness': 2,'valence': 2, 'tempo': 2})
features2.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,length_in_minutes,time_signature
0,0.375,0.728,A,-10.130,minor,0.0433,0.759,0.860000,0.1410,0.9010,118.414,audio_features,7A80n2HvAs1rZNNs64NN8F,2.96,4/4
1,0.776,0.659,A,-7.767,major,0.0510,0.434,0.000000,0.0868,0.9690,123.094,audio_features,5z8DiKSG4EwxmO6gUi12rZ,2.67,4/4
2,0.426,0.300,B,-9.717,minor,0.0365,0.573,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,3.50,4/4
3,0.426,0.300,B,-9.717,minor,0.0365,0.573,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,3.50,4/4
4,0.555,0.293,D#/Eb,-8.525,minor,0.0470,0.877,0.000001,0.1180,0.1130,140.935,audio_features,54a3QojIm00P4MXL02BZvC,1.89,3/4
5,0.614,0.277,A,-12.052,major,0.1200,0.852,0.000000,0.1060,0.5810,122.659,audio_features,1ebslsQcXFPNRBLsa76qEK,3.00,4/4
6,0.137,0.119,C#/Db,-23.390,major,0.0575,0.978,0.940000,0.1010,0.0374,70.023,audio_features,41F8XFf9aYq1vCMqyuLNFl,5.60,4/4
7,0.818,0.151,A,-21.946,minor,0.0718,0.917,0.917000,0.1110,0.4140,142.153,audio_features,6OzRUo7b23kXFXHWLpigCj,2.03,4/4
8,0.566,0.237,F#/Gb,-17.012,minor,0.0282,0.398,0.897000,0.1160,0.0761,80.020,audio_features,61liR3eWrA8BTeUnltA32e,2.45,4/4
9,0.694,0.337,F,-12.300,minor,0.0624,0.609,0.597000,0.1760,0.5690,126.235,audio_features,601ppwJ942tjJ9aDPGYwOT,3.17,4/4


round values to 2 decimal places

In [6]:
features2 = features2.round({'danceability': 2, 'energy': 2,'loudness': 2,'speechiness': 2,'acousticness': 2,'instrumentalness': 2,'liveness': 2,'valence': 2, 'tempo': 2})
features2.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,length_in_minutes,time_signature
0,0.38,0.73,A,-10.13,minor,0.04,0.76,0.86,0.14,0.90,118.41,audio_features,7A80n2HvAs1rZNNs64NN8F,2.96,4/4
1,0.78,0.66,A,-7.77,major,0.05,0.43,0.00,0.09,0.97,123.09,audio_features,5z8DiKSG4EwxmO6gUi12rZ,2.67,4/4
2,0.43,0.30,B,-9.72,minor,0.04,0.57,0.00,0.13,0.52,116.44,audio_features,7hkyFS309jQBrwrbxGsJuH,3.50,4/4
3,0.43,0.30,B,-9.72,minor,0.04,0.57,0.00,0.13,0.52,116.44,audio_features,7hkyFS309jQBrwrbxGsJuH,3.50,4/4
4,0.56,0.29,D#/Eb,-8.52,minor,0.05,0.88,0.00,0.12,0.11,140.94,audio_features,54a3QojIm00P4MXL02BZvC,1.89,3/4
5,0.61,0.28,A,-12.05,major,0.12,0.85,0.00,0.11,0.58,122.66,audio_features,1ebslsQcXFPNRBLsa76qEK,3.00,4/4
6,0.14,0.12,C#/Db,-23.39,major,0.06,0.98,0.94,0.10,0.04,70.02,audio_features,41F8XFf9aYq1vCMqyuLNFl,5.60,4/4
7,0.82,0.15,A,-21.95,minor,0.07,0.92,0.92,0.11,0.41,142.15,audio_features,6OzRUo7b23kXFXHWLpigCj,2.03,4/4
8,0.57,0.24,F#/Gb,-17.01,minor,0.03,0.40,0.90,0.12,0.08,80.02,audio_features,61liR3eWrA8BTeUnltA32e,2.45,4/4
9,0.69,0.34,F,-12.30,minor,0.06,0.61,0.60,0.18,0.57,126.24,audio_features,601ppwJ942tjJ9aDPGYwOT,3.17,4/4


## Serialize transformed data to a flat file. 

I did multiple versions because I wasn't sure which would be easier to use in later steps.

In [7]:
weather_imperial.to_csv('transformed_weather.csv')
weather_imperial.to_json('transformed_weather.json')
weather_imperial.to_json('transformed_weather2.json', orient="index")


In [ ]:

features2.to_csv('transformed_audio_features.csv')
features2.to_json('transformed_audio_features.json')
features2.to_json('transformed_audio_features2.json', orient="index")


In [ ]:

tracksdf2.to_csv('transformed_stream_hist.csv')
tracksdf2.to_json('transformed_stream_hist.json')
tracksdf2.to_json('transformed_stream_hist2.json', orient="index")
